# Tutorial to check the structures of all S2/S3 products with xarray/datatree

In [ ]:
from IPython.display import display
from pathlib import Path
from xarray_eop.eop import open_eop_datatree

### Get AWS credentials

In [ ]:
import json
# Assuming credentials are in ~/.eopf/secrets.json
try:
    SECRET_PATH = Path.home() / Path(".eopf/secrets.json")
    with open(SECRET_PATH) as f:
        secrets=json.load(f)

# Or use ENV variables
except:
    import os
    secrets = {"s3input" : {
        "key": os.getenv("AWS_ACCESS_KEY_ID"),
        "secret": os.getenv("AWS_SECRET_ACCES_KEY"),
        "endpoint_url": os.getenv("AWS_ENDPOINT_URL"),
        "region_name": os.getenv("AWS_DEFAULT_REGION")
        }
    }

secrets["s3input"].pop("region_name",None)


### Opening the products with datatree

Local storage

In [ ]:
# LOCAL_PATH = Path("/mount/internal/work-st/projects/cs-412/2078-dpr/Samples/Products/Zarr_Beta")
LOCAL_PATH = Path("/mount/internal/work-st/projects/cs-412/2078-dpr/Deliveries/DR_S3_20240308/Products/")
ZIPPED=True
if ZIPPED:
    local_store = LOCAL_PATH.glob("S3*.zip")
    products = [f"zip::{str(p)}" for p in local_store if "L0" not in str(p) and "CR0" not in str(p)]
else:
    local_store = LOCAL_PATH.glob("S3*.zip")
    products = [ p for p in local_store if "L0" not in str(p) and "CR0" not in str(p)]
products

In [ ]:
for prod in products:
    print(prod)
    dt = open_eop_datatree(prod,decode_times=False)
    display(dt)

Cloud storage

In [ ]:
import s3fs
SAMPLE_PATH = "s3://s3-input/Products/NewFormat/"
s3 = s3fs.S3FileSystem(
    key=secrets["s3input"]["key"],
    secret=secrets["s3input"]["secret"],
    endpoint_url=secrets["s3input"]["endpoint_url"]
)
s3_path = s3.glob(SAMPLE_PATH+"S3*.zip")
stores = [ f"zip::s3://{p}" for p in s3_path if "L0" not in p and "CR0" not in p]
stores

In [ ]:
for prod in stores:
    print(prod)
    dt = open_eop_datatree(prod,decode_times=False,backend_kwargs={"storage_options": {"s3":secrets["s3input"]}})
    display(dt)